In [10]:
%pip install tqdm

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [11]:
from transformers import AutoTokenizer, pipeline, PretrainedConfig
from optimum.onnxruntime import ORTModelForSeq2SeqLM
import onnxruntime

In [15]:
# Load encoder model
encoder_session = onnxruntime.InferenceSession('quantized_oyto_t5_small_onnx/encoder_model_quantized.onnx')

# Load decoder model
decoder_session = onnxruntime.InferenceSession('quantized_oyto_t5_small_onnx/decoder_model_quantized.onnx')

In [16]:
model_id = "quantized_oyto_t5_small_onnx"
tokenizer = AutoTokenizer.from_pretrained(model_id)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [17]:
config = PretrainedConfig.from_json_file('quantized_oyto_t5_small_onnx/config.json')

In [18]:
model = ORTModelForSeq2SeqLM(
    config=config,
    onnx_paths=['quantized_oyto_t5_small_onnx/decoder_model_quantized.onnx','quantized_oyto_t5_small_onnx/encoder_model_quantized.onnx'],
    encoder_session=encoder_session, 
    decoder_session=decoder_session, 
    model_save_dir='quantized_oyto_t5_small_onnx',
    use_cache=False, 
)

In [19]:
onnx_translation = pipeline("translation_unyo_to_dcyo", model=model, tokenizer=tokenizer)

In [13]:
text_that_work_1 = "Bi a ba lo osuwon iseda, gbogbo gbolohun ede Yoruba ni a le pin si meji ni gbooro bayii"
text_that_work_2 = "Nje omo naa ni owo?"

In [20]:
# olorun oba o, ese o
text = "Nje omo naa ni owo?" # mo lo si ile
result = onnx_translation(text, max_length = 10000)
result

[{'translation_text': 'Ǹjẹ́ ọmọ náà ní owó?'}]